<a href="https://colab.research.google.com/github/Peeyusj/makeMorePart2/blob/main/week6_mlp_minibatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Week 6: MLP with Mini-batches
# Following Karpathy's Makemore Part 2

import torch
import torch.nn.functional as F
import pandas as pd